In [6]:
import cv2
import mediapipe as mp
import math
import time
import alsaaudio

In [8]:
# Initialize video capture
cap = cv2.VideoCapture(0)

# Initialize MediaPipe hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_drawing = mp.solutions.drawing_utils

# Previous time for FPS calculation
pTime = 0

# Define fixed height and width of the rectangle
RECT_HEIGHT = 200
RECT_WIDTH = 50

# Define fixed position of the rectangle
RECT_X = 50
RECT_Y = 150

# Function to map a value from one range to another
def map_range(value, from_low, from_high, to_low, to_high):
    return to_low + (to_high - to_low) * ((value - from_low) / (from_high - from_low))

while True:
    # Read frame from camera
    ret, frame = cap.read()

    # Flip frame horizontally
    frame = cv2.flip(frame, 1)

    # Calculate FPS
    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime

    # Display FPS on the frame
    text = f"FPS: {str(int(fps))}"
    cv2.putText(frame, text, (10, 70), cv2.FONT_HERSHEY_PLAIN, 3, (255, 0, 255), 3)

    # Convert frame to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame to detect hand landmarks
    results = hands.process(rgb_frame)

    # If hand landmarks are detected
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Get coordinates of thumb tip and index fingertip
            thumb_top = hand_landmarks.landmark[4]
            index_fingertip = hand_landmarks.landmark[8]
            height, width, _ = frame.shape

            # Calculate fingertip coordinates
            cx1, cy1 = int(index_fingertip.x * width), int(index_fingertip.y * height)
            cv2.circle(frame, (cx1, cy1), 10, (0, 255, 0), -1) 

            # Calculate thumb tip coordinates
            cx2, cy2 = int(thumb_top.x * width), int(thumb_top.y * height)
            cv2.circle(frame, (cx2, cy2), 10, (0, 255, 0), -1)

            # Calculate midpoint between thumb tip and index fingertip
            mid_x = (cx1 + cx2) // 2
            mid_y = (cy1 + cy2) // 2
            cv2.circle(frame, (mid_x, mid_y), 10, (0, 0, 255), -1)

            # Make a line between them
            cv2.line(frame, (cx1, cy1), (cx2, cy2), (255,0,0), 5)

            # Calculate distance between thumb tip and index fingertip
            distance = math.sqrt((cx2 - cx1) ** 2 + (cy2 - cy1) ** 2)
            cv2.putText(frame, str(int(distance)), (250, 70), cv2.FONT_HERSHEY_PLAIN, 3, (0, 0, 255), 3)

            # Map distance to fill of rectangle
            rect_fill = int(map_range(distance, 12, 220, 0, RECT_HEIGHT))
            if rect_fill > RECT_HEIGHT:
                rect_fill = RECT_HEIGHT

            # Draw fixed rectangle with fill mapped to distance
            per_map = int(map_range(distance, 12, 220, 0, 100))
            if per_map > 100:
                per_map = 100
            elif per_map < 0:
                per_map = 0
            cv2.rectangle(frame, (RECT_X, RECT_Y), (RECT_X + RECT_WIDTH, RECT_Y + RECT_HEIGHT), (255, 0, 0), 3)
            cv2.rectangle(frame, (RECT_X, RECT_Y + RECT_HEIGHT - rect_fill), (RECT_X + RECT_WIDTH, RECT_Y + RECT_HEIGHT), (255, 0, 0), thickness=cv2.FILLED)
            per = f"{str(int(per_map))}%"
            cv2.putText(frame, per, (RECT_X, RECT_Y-3), cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
            
            # Pulse audio mapping and implementation
            audio_map = int(map_range(distance, 10, 220, 0, 100))
            if audio_map > 100:
                audio_map = 100
            elif audio_map < 0:
                audio_map = 0
            m = alsaaudio.Mixer()
            new_volume = audio_map
            m.setvolume(new_volume)
            
                    
            # # Draw extra hand landmarks
            # mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    # Display frame
    cv2.imshow("frame", frame)

    # Check for key press to exit
    key = cv2.waitKey(1)
    if key == ord('q'):
        break

# Release video capture and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


I0000 00:00:1714733366.974525   24105 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1714733366.975909   33355 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.0.4-arch1.1), renderer: Mesa Intel(R) HD Graphics 4400 (HSW GT2)
